In [ ]:
#!pip install langchain==0.0.163
#!pip install pygpt4all==1.1.0
#!pip install transformers
#!pip install datasets
#!pip install chromadb
#!pip install tiktoken

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

def text_embedding(text):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(text, normalize_embeddings = True)

def vector_similarity(vec1, vec2):
    return np.dot(np.squeeze(np.array(vec1)),np.squeeze(np.array(vec2)))

phrase1    = "Apple is a fruit"
embedding1 = text_embedding(phrase1)
print(len(embedding1))

phrase2    = "Apple iPhone is expensive"
embedding2 = text_embedding(phrase2)
print(len(embedding2))

phrase3    = "Mango is a fruit"
embedding3 = text_embedding(phrase3) 
print(len(embedding3))

phrase4    = "There is a new Apple iPhone"
embedding4 = text_embedding(phrase4)
print(len(embedding4))

print(vector_similarity(embedding1,embedding3))
print(vector_similarity(embedding1,embedding4))

print(vector_similarity(embedding2,embedding3))
print(vector_similarity(embedding2,embedding4)) 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\gsdav\Anaconda3\envs\llama_local\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gsdav\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384
384
384
384
0.67738634
0.38097987
0.15007739
0.6433085


In [ ]:
!conda info

In [3]:
import pandas as pd
from datasets import load_dataset

# Download the medical_dialog dataset from Hugging Face
dataset = load_dataset('medical_dialog', 'processed.en')

# Convert the dataset to a pandas dataframe
df = pd.DataFrame(dataset['train'])

# Print the first 5 rows of the dataframe
df.head()

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dialog = []
# make each sentence on a seperate row
patient, doctor = zip(*df['utterances'])
for i in range(len(patient)):
  dialog.append(patient[i])
  dialog.append(doctor[i])

dialog_df = pd.DataFrame({"dialog": dialog})
# save the data to txt file
dialog_df.to_csv('data.txt', sep=' ', index=False)

In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings



In [10]:


# Path to your local model
local_model_path = r'C:\Users\gsdav\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2\snapshots\84f2bcc00d77236f9e89c8a360a00fb1139bf47d'

# Instantiate the HuggingFaceEmbeddings class
# embedding = HuggingFaceEmbeddings(model_name=local_model_path)
# Instantiate the HuggingFaceEmbeddings class
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


# Use the embedding instance
# documents = ["foo bar"]
# output = embedding.embed_documents(documents)

In [6]:
# add the path to the CV as a PDF
loader = TextLoader('data.txt')
# Embed the document and store into chroma DB
index = VectorstoreIndexCreator(embedding= HuggingFaceEmbeddings()).from_loaders([loader])

In [7]:
# specify the path to the .bin downloaded file
# local_path = './models/ggml-gpt4all-j-v1.3-groovy.bin'  # replace with your desired local file path
local_path = 'C:/Users/gsdav/AppData/Local/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf'
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True, backend='gptj')

In [11]:
# perform similarity search and retrieve the context from our documents
results = index.vectorstore.similarity_search("what is the solution for sore back", k=4)
# join all context information (top 4) into one string 
context = "\n".join([document.page_content for document in results])
print(f"Retrieving information related to your question...")
print(f"Found this content which is most similar to your question: {context}")

Retrieving information related to your question...
Found this content which is most similar to your question: "patient: i had suffered with harpez and pneumonia. i had taken zovirax intravenous depomedrol,tricot xylocaine and finally seftum 500 mg. the eruptions and irritable skin in groin and arm pits. my face looked i was older than my age. one day out of a miracle i had formulated my own drug which has saved me. the drug is powdered ammoxycilin 500mg, seftum500mg, althrocin 500 mg, cifran 500mgin right proportions. i applied to my groin, face, armpits. also like homeopathy i tried taking one milligram of this mixture rubbed on epiglottis. my life style has changed completely and would like to recommend to this entire world that this is a miracle drug that can keep in abeyance many diseases of the world. no side effects. excellent health, looking 30 years younger with no damage to skin and any side effects, b.shankar sharma, mail address yyyy@yyyy"
"doctor: nsaids. otc nsaids can hel

In [13]:
template = """
Please use the following context to answer questions.
Context: {context}
---
Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)

llm_chain = prompt | llm
# Print the result
print("Processing the information with gpt4all...\n")
print(llm_chain.invoke("what is the best advice for staying healthy?"))

Processing the information with gpt4all...

 First, we have to understand that this coronavirus is a highly contagious virus and can spread through contact with an infected person or contaminated surfaces. So, hand hygiene is crucial. Wash your hands frequently with soap and water for at least 20 seconds especially after blowing your nose, coughing or sneezing; before eating; and after using the bathroom.

If you don't have access to soap and water, use a sanitizer that contains at least 60% alcohol. And remember, it's not just about washing your hands but also keeping surfaces clean by wiping them down with disinfectant wipes regularly.

Also, avoid touching your face especially your eyes, nose, and mouth as much as possible because these are the most common entry points for viruses to enter our bodies.

And finally, stay informed through credible sources like WHO or CDC about any updates on COVID-19. Stay home if you're feeling unwell, and follow proper quarantine procedures if neces